In [1]:
import numpy as np
import matplotlib.pyplot as plt
from random import shuffle
from sklearn.datasets import make_multilabel_classification
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.tree import DecisionTreeClassifier

In [24]:
n_instancias=10
n_caracteristicas=8
n_clases=5

In [71]:
def conjunto_datos(n_instancias,n_caracteristicas,n_clases):
        return make_multilabel_classification(n_samples=n_instancias, 
                                              n_features=n_caracteristicas, n_classes=n_clases)
         

In [129]:
X,Y=conjunto_datos(n_instancias,n_caracteristicas,n_clases)

In [79]:
def aleatorio_boolean(n_caracteristicas):
        rnd_dimensions=np.random.randint(0, 2,n_caracteristicas)
        return rnd_dimensions.astype(bool)
        

In [86]:
def aleatorio_array(n_instancias):
        s=list(range(n_instancias))
        shuffle(s)
        return np.array(s[:(int(len(s)/2))])

In [95]:
def reducir_datos(X,rnd_dimensions):
        return X[:, rnd_dimensions]

In [108]:
def vecino_cercano(n_instancias,rnd_neighbors,m_reducida):
    m_vecinos=np.zeros((instancias,len(rnd_neighbors)))
    cont=-1
    for i in m_reducida:
        dist=999
        cont+=1
        cont2=-1
        for j in rnd_neighbors:
            cont2+=1
            dist2=euclidean_distances([i],[m_reducida[j,:]])
            if dist2<dist:
                dist=dist2
                a=cont
                b=cont2
        m_vecinos[a][b]=1
    return m_vecinos

In [117]:
def entrenamiento(m_vecinos):
    return np.concatenate((X,m_vecinos),axis=1)

In [126]:
def clasificador(m_entrenamiento,Y):
    e=DecisionTreeClassifier()
    return e.fit(m_entrenamiento,Y)

In [132]:
def fit(X,Y):
    rnd_dimensions=aleatorio_boolean(n_caracteristicas)
    rndneighbors=aleatorio_array(n_instancias)
    m_reducida=reducir_datos(X,rnd_dimensions)
    m_vecinos=vecino_cercano(n_instancias,rnd_neighbors,m_reducida)
    m_entrenamiento=entrenamiento(m_vecinos)
    return clasificador(m_entrenamiento,Y)

In [133]:
fit(X,Y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [134]:
m_predecir=np.zeros((len(rnd_neighbors),caracteristicas))
cont=0
for i in range(len(X)):
    if i in rnd_neighbors:
        m_predecir[cont]=X[i,:]
        cont+=1
m_predecir

array([[  5.,   8.,   1.,   4.,  10.,   2.,   5.,   2.],
       [  9.,   0.,   3.,   2.,  10.,   1.,   5.,   4.],
       [ 14.,   0.,   9.,  13.,  11.,   1.,   4.,  10.],
       [  5.,   7.,   4.,   6.,   3.,   7.,   4.,   6.],
       [  4.,   7.,  10.,  10.,   9.,   6.,   2.,   8.]])

In [539]:
e.predict(X)
print(e)

ValueError: Number of features of the model must match the input. Model n_features is 13 and input n_features is 8 